In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import plotly.graph_objects as go
import cv2
import os
import plotly.io as pio

os.chdir("/Users/pwecker/dev/DataVizDemocracy")

In [ ]:
def extend_df_by_continents(df):
    continent_map_df = pd.read_csv('data/country_continent_map.csv')[["iso_code", "continent"]].drop_duplicates()
    return pd.merge(df, continent_map_df, left_on="Code", right_on="iso_code")

def extend_df_by_population(df):
    pop_df = pd.read_csv("data/life-expectancy-vs-liberal-democracy-index.csv")[["Code", "Year", "Population (historical)"]]   
    return (pd.merge(df, pop_df, on=["Code", "Year"]).rename({"Population (historical)":"pop"}, axis=1))

def extend_df_by_edi(df):
    edi_df = (pd.read_csv("data/electoral-democracy-index.csv")
            .rename({"Electoral democracy index (best estimate, aggregate: average)": "edi"}, axis=1))
    return pd.merge(df, edi_df, on=["Code", "Year"])

# CO2

In [ ]:
df = pd.read_csv("/Users/pwecker/dev/DataVizDemocracy/data/emission/data.csv", sep=",")
df_melted = pd.melt(df,
                    id_vars=list(df.columns[:4]),
                    value_vars=list(df.columns[4:-1]),
                    var_name="Year",
                    value_name="CO2")
df_melted = (df_melted[["Country Code", "Year", "CO2"]]
            .rename(columns={"Country Code":"Code"})
            .dropna()
            .astype({'CO2': 'int64',
                     "Year": "int64",
                     "Code": "str"}))
co2_df = df_melted[df_melted["CO2"]>=0]

co2_df = extend_df_by_continents(co2_df)
co2_df = extend_df_by_population(co2_df)
co2_df = extend_df_by_edi(co2_df)
co2_df.head(1)


# Religion

In [ ]:
rel_df = pd.read_csv("data/religion/national_data.csv") # as found here: https://data.world/cow/world-religion-data
codes_cow_df = pd.read_csv("./data/religion/final_merged_country_data_v2.csv")[["Code", "CCode"]]
pd.options.mode.copy_on_write = True
main_rels = ["judgenpct", "islmgenpct", "chrstgenpct", "budgenpct", "hindgenpct", "sikhgenpct", "shntgenpct", "bahgenpct", "taogenpct", "confgenpct","anmgenpct"]
main_rels_abs = [rel[:-3] for rel in main_rels] # get "chrstgen" for "chrstgenpct"

main_rels_df = rel_df[["year", "state", "total", "nonreligpct", "othrgenpct"] + main_rels + main_rels_abs]
# Ensure all values in the main religion columns are numeric and replace NaNs with zeros
main_rels_df[main_rels] = main_rels_df[main_rels].apply(pd.to_numeric, errors='coerce').fillna(0)

# Replace NaNs in 'total' with 1 to avoid division by zero
main_rels_df['total'] = pd.to_numeric(main_rels_df['total'], errors='coerce').fillna(1)
main_rels_df[main_rels] = main_rels_df[main_rels].div(main_rels_df['total'], axis=0)
main_rels_df['primary_rel'] = main_rels_df[main_rels].idxmax(axis=1)
main_rels_df[main_rels] = main_rels_df[main_rels].apply(pd.to_numeric, errors='coerce')
main_rels_df[main_rels] = main_rels_df[main_rels].fillna(0)

def calculate_entropy(row, main_rels):
    try:
        proportions = row[main_rels].values / 100  # Convert percentage values to proportions
        proportions = proportions[proportions > 0]  # Filter out zero proportions to avoid log(0)
        if len(proportions) == 0:
            return 0  # If no proportions are greater than zero, entropy is zero
        proportions = proportions.astype(float)  # Ensure all values are floats
        entropy = -np.sum(proportions * np.log2(proportions))  # Calculate entropy
        return entropy
    except Exception as e:
        print(f"Error calculating entropy for row: {row.name}, error: {e}")
        return np.nan

# Apply the entropy calculation to each row
main_rels_df['entropy'] = main_rels_df.apply(calculate_entropy, axis=1, main_rels=main_rels)

main_rels_df = main_rels_df.rename({"state":"CCode", "year": "Year"}, axis=1)
rel_df = pd.merge(main_rels_df, codes_cow_df.drop_duplicates(), on="CCode", )
rel_df = rel_df[["Code", "Year", "nonreligpct", "entropy", "primary_rel"] + main_rels_abs]
rel_df = extend_df_by_edi(rel_df)
rel_df = extend_df_by_continents(rel_df)
rel_df = extend_df_by_population(rel_df)
main_rels_comp_pct = [rel+"comppct" for rel in main_rels_abs]
rel_df[main_rels_comp_pct] = rel_df[main_rels_abs].div(rel_df[main_rels_abs].sum(axis=1), axis=0)
# Compute HHI
rel_df['HHI'] = rel_df[main_rels_comp_pct].apply(lambda x: sum(x**2), axis=1)

# Map

In [ ]:
map_df = gpd.read_file('data/geodata/World_Countries.shp')
map_df = map_df.set_index('COUNTRY')

# Plot Functions

In [ ]:
def plot_edi_map(edi_df, map_df, title, year, col="edi", use_log_scale=False, save_dont_show=False, folder="co2_map_plots", bottom_x = None, top_x=None):
    # Filter and clean the data
    edi_df = edi_df.set_index('Entity')
    edi_df = edi_df.dropna(how='all')
    edi_df = edi_df[edi_df['Year'] == year]
    
    # Merge the map data with the edi data
    edi_map_df = map_df.join(edi_df, how='left')
    
    if use_log_scale:
        # Apply logarithmic transformation
        edi_map_df[col] = edi_map_df[col].apply(lambda x: np.log10(x) if x > 0 else np.nan)
        if top_x is not None:
            top = top_x
        else:
            top = edi_map_df[col].max()
        if bottom_x is not None:
            bottom = bottom_x
        else:
            bottom=edi_map_df[col].min()
        norm = plt.Normalize(vmin=bottom, vmax=top)

        cbar_label = f'Log10({col})'
    else:
        if top_x is not None:
            top = top_x
        else:
            top = edi_map_df[col].max()
        if bottom_x is not None:
            bottom = bottom_x
        else:
            bottom=edi_map_df[col].min()
        norm = plt.Normalize(vmin=bottom, vmax=top)

        cbar_label = col
    
    # Create the plot
    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    map_df.plot(color='grey', ax=ax)
    
    # Plot the data with the specified column and color map
    p = edi_map_df.plot(column=col, 
                        cmap='plasma', 
                        legend=False, 
                        norm=norm,
                        ax=ax)
    
    ax.axis('off')
    ax.set_title(f'{title} {year}')
    
    # Add color gradient legend in custom axis
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="10%", pad=0.1)
    
    # Use the ScalarMappable to ensure the colorbar matches the plot
    sm = plt.cm.ScalarMappable(cmap='plasma', norm=norm)
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')
    cbar.set_label(cbar_label)

    plt.tight_layout()
    if save_dont_show:
        plt.savefig(f"./{folder}/{title} {year}")
    else:
        plt.show()


In [ ]:
def zabi_scatter(df: pd.DataFrame, x: str, y: str, year: int, size: str, color: str, title: str, uselogx=False, uselogy=False, savedontshow=False, location=""):
    if year != 0:
        df = df[df["Year"]==year]
    if size is not None:
        df[size] = df[size].apply(lambda x: 60000000 if x<60000000 else x) / 1e6
    fig = px.scatter(
    df,
    x=x,
    y=y,
    size=size,  # Use the scaled population
    color=color,
    hover_name='Entity',
    size_max=40, # Increase size_max to make points larger -- 200 worked the best
    title=f"{title} {year}",
    )

    # Update the layout
    if uselogx and uselogy:
        fig.update_layout(
            xaxis_title=x,
            yaxis_title=y,
            legend_title=color,
            template='plotly',
            height=800,
            yaxis_type="log",
            xaxis_type="log"
        )
    elif uselogx:
        fig.update_layout(
            xaxis_title=x,
            yaxis_title=y,
            legend_title=color,
            template='plotly',
            height=800,
            xaxis_type="log"
        )
    elif uselogy:
        fig.update_layout(
            xaxis_title=x,
            yaxis_title=y,
            legend_title=color,
            template='plotly',
            height=800,
            yaxis_type="log"
        )        

    # Show the plot
    if savedontshow:
        pio.write_html(fig, f"/Users/pwecker/dev/DataVizDemocracy/plots_paul/relig_scatter_plots/{title}_{year}_interactive.html")
    else:
        fig.show()


# CREATING PLOTS

In [ ]:
plot_edi_map(rel_df, map_df, "Religous Mixedness", 2000, col="HHI")

In [ ]:
for year in list(rel_df["Year"].unique()):
    plot_edi_map(rel_df, map_df, "Religious Diversity", year, col="HHI", save_dont_show=True, folder="plots_paul/relig_map_plots")

In [ ]:
plot_edi_map(rel_df, map_df, "Non-Religousness", 2010, col="nonreligpct")

In [ ]:
for year in list(co2_df["Year"].unique()):
    plot_edi_map(co2_df, map_df, "CO2 Emission", year, col="CO2", save_dont_show=True,
                 folder="plots_paul/co2_map_plots", use_log_scale=True, bottom_x=3, top_x=8)

In [ ]:
for year in list(rel_df["Year"].unique()):
    print(int(year))
    zabi_scatter(rel_df, "edi", "HHI", 
                year=int(year), size="pop", 
                color= "continent", 
                title="Electoral Democracy Index vs. Religious Diversity",
                uselogy=False, savedontshow=True, location="relig_scatter_plots")

In [ ]:
list(rel_df["Year"].unique())

In [ ]:
for year in list(rel_df["Year"].unique()):
    zabi_scatter(rel_df, "edi", "nonreligpct", 
                year=int(year), size="pop", 
                color= "continent", 
                title="Electoral Democracy Index vs. Non-Religious",
                uselogy=True, savedontshow=True, location="relig_scatter_plots")


In [17]:
zabi_scatter(co2_df, "edi", "CO2", title="Electoral Democracy Index vs. CO2 Emissions",
             year=2000, size="pop", color="continent", uselogy=True)


In [ ]:
for year in list(co2_df["Year"].unique()):
    zabi_scatter(co2_df, "edi", "CO2", title="Electoral Democracy Index vs. CO2 Emissions", year=int(year), size="pop", color="continent", uselogy=True,
                 location="co2_scatter_plots", savedontshow=True)


In [12]:
co2_rel_df = pd.merge(co2_df, rel_df[["Code", "Year", "nonreligpct", "primary_rel", "HHI"]], on=["Code", "Year"])

In [13]:
zabi_scatter(co2_rel_df, "CO2", "HHI", title="CO2 Emissions vs. Religous Mixedness", year = 2010, size="pop", color="primary_rel", uselogx=True)

In [ ]:
for year in list(co2_rel_df["Year"].unique()):
    zabi_scatter(co2_rel_df, "CO2", "HHI", title="CO2 Emissions vs. Religous Mixedness", year = int(year), size="pop", color="primary_rel", uselogx=True,
                 location="co2_rel_scatters", savedontshow=True)

    

In [16]:
co2_rel_df.to_pickle("./data/co2_rel_df.pkl")
rel_df.to_pickle("./data/rel_df.pkl")
co2_df.to_pickle("./data/co2_df.pkl")

In [ ]:
# Apply ggplot style
plt.style.use('ggplot')

# Get unique continents
continents = rel_df['continent'].unique()

# Create subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 15))
axes = axes.flatten()

# Add box plots for each continent
for i, continent in enumerate(continents):
    ax = axes[i]
    filtered_df = rel_df[rel_df['continent'] == continent]
    filtered_df.boxplot(column='nonreligpct', by='Year', ax=ax)
    ax.set_title(continent)
    ax.set_xlabel('Year')
    ax.set_ylabel('Non-Religious Percentage')
    ax.grid(True)

# Adjust layout and remove superfluous titles
fig.suptitle('Non-Religious Percentage Distribution by Year for Each Continent', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:


# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Add box plot for non-religious percentage by year, ignoring continents
rel_df.boxplot(column='nonreligpct', by='Year', ax=ax)

# Set titles and labels
ax.set_title('Non-Religious Percentage Distribution by Year (All Continents)')
ax.set_xlabel('Year')
ax.set_ylabel('Non-Religious Percentage')
ax.grid(True)

# Remove the automatic 'by' title
plt.suptitle('')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
def make_video(image_folder:str, video_name:str):
    image_folder = image_folder
    video_name = video_name + ".avi"
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort()
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')  # You can use other codecs like 'XVID', 'MJPG', 'X264', etc.
    video = cv2.VideoWriter(video_name, fourcc, 3, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))
    video.release()
    cv2.destroyAllWindows()

In [ ]:
make_video(image_folder="relig_map_plots", video_name="evolution_of_religious_diversity")

In [ ]:
make_video(image_folder="co2_map_plots", video_name="co2_evolution")

In [ ]:
continent_co2 = co2_df[["Year", "CO2", "continent", "pop"]].groupby(by=["Year", "continent"]).agg({"CO2": "sum", "pop": "sum"}).reset_index()
continent_co2

In [ ]:
# Group by Year and continent, summing the CO2 and pop columns
plt.style.use('ggplot')
continent_co2 = co2_df[["Year", "CO2", "continent", "pop"]].groupby(by=["Year", "continent"]).agg({"CO2": "sum", "pop": "sum"}).reset_index()

# Pivot the DataFrame to have years as index and continents as columns
pivot_df = continent_co2.pivot(index='Year', columns='continent', values='CO2').fillna(0)

# Plotting
fig, ax = plt.subplots()

# Define the width of the bars
width = 0.5

# Create an array of zeros for the bottom positions
bottom = np.zeros(len(pivot_df))

# Plot each continent
for continent in pivot_df.columns:
    ax.bar(pivot_df.index, pivot_df[continent], width, label=continent, bottom=bottom)
    bottom += pivot_df[continent]

# Set the title and legend
ax.set_title("CO2 Emissions by Continent and Year")
ax.set_xlabel("Year")
ax.set_ylabel("CO2 Emissions")

ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.show()
fig.savefig("plots_paul/CO2 Emissions by Continent and Year.png")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def bar_plots(df: pd.DataFrame, x: str, y: str, color: str, title: str, width:float =.5, location: str = "", savedontshow: bool = False, xaxislog: bool = False, yaxislog: bool = False, agg = "sum"):
    """
    Create one stack of multicolored bars with the height of x for each value in y.
    Arguments:
        df: dataframe with data. Should have columns x, y, color
        x: column which accounts for the height of the bars
        y: for each value in y, compute one stack of bars
        color: color for each subbar
        title: title seen on plot, and title for saving
        location: folder in which image should be saved if so
        savedontshow: specifies if figure should be saved instead of displayed. should be saved at given location
        xaxislog: whether x axis should be of logarithmic scale
        yaxislog: whether y axis should be of logarithmic scale
    """
    # Apply ggplot style
    plt.style.use('ggplot')

    # Group by y and color, summing the x column
    grouped_df = df.groupby(by=[y, color]).agg({x: agg}).reset_index()

    # Pivot the DataFrame to have y as index and color as columns
    pivot_df = grouped_df.pivot(index=y, columns=color, values=x).fillna(0)

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # Define the width of the bars
    width = width

    # Create an array of zeros for the bottom positions
    bottom = np.zeros(len(pivot_df))

    # Plot each color category
    for category in pivot_df.columns:
        ax.bar(pivot_df.index, pivot_df[category], width, label=category, bottom=bottom)
        bottom += pivot_df[category]

    # Set the title and labels
    ax.set_title(title)
    ax.set_xlabel(y)
    ax.set_ylabel(x)

    # Set logarithmic scale if specified
    if xaxislog:
        ax.set_xscale('log')
    if yaxislog:
        ax.set_yscale('log')

    # Move the legend outside the plot
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    # Adjust layout
    plt.tight_layout()

    # Save or show the plot
    if savedontshow:
        plt.savefig(f"{location}/{title}.png")
    else:
        plt.show()

# Example usage:
bar_plots(rel_df, x='nonreligpct', y='Year', color='continent', title='Non-Religion by Continent and Year', width=3, savedontshow=True, location='plots_paul', agg =  "mean")


In [ ]:
bar_plots(rel_df, x='', y='Year', color='continent', title="CO2 Emissions by Continent and Year", width=.5, savedontshow=True, location='plots_paul', agg =  "mean")
